In [ ]:
# Disclaimer: I always refer to themachinethatgoesping as 'Ping' in my comments

from matplotlib import pyplot as plt
import numpy as np

# progressbars (sorry I love this one)
from tqdm.auto import tqdm

# this has the kongsberg all filehandler
from themachinethatgoesping.echosounders import kongsbergall

# ping filtering
from themachinethatgoesping.echosounders import pingtools

# this contains a function extract the water column image to a depth/x in meters
import themachinethatgoesping.pingprocessing as pingprocessing



In [ ]:
import themachinethatgoesping
print (themachinethatgoesping.version())

In [ ]:
file = '../unittest_data/kongsberg/g/-7041029013895133878.wcd'

In [ ]:
# create the filehandler, this will open the filestream, index the file and initialze the ping interface
# Note (1):If your file is big this can take some seconds and you may want to use Ping's the caching mechanism
# In this case: ask me about file caching
# Note (2): The filehandler can also handle a whole list of files, the get_pings() function will then give you all pings of all files (as if it would be a single filestream)
# If you don't want this function to be verbose set 'show_progress' to False
filehandler = kongsbergall.KongsbergAllFileHandler(file, show_progress=True)

In [ ]:
#extract all pings as a list
#Note: a pingobject does not contain much data (only auxiliary information)
# You can thus copy
# The image is read from the file, the moment you call watercolumn.get_amplitudes or watercolumn.get_av 
# pings are random access
pings = filehandler.get_pings()

print('Number of pings found in file:', len(pings))

pings_with_watercolumn = pingprocessing.filter_pings.by_features(pings, ['watercolumn.av'])
print('Number of pings found in file with watercolumn data:', len(pings_with_watercolumn))

images_beam_sample = []
images_depths_y = []

#use slicing to loop over the first 10 pings
# You could also do something like pings_with_watercolumn[::2] to loop over every 2nd image etc
for ping in tqdm(pings_with_watercolumn[:10], 'Reading pings'):
    #check if the ping has watercolumn amplitude data
    if ping.has_watercolumn():
        # extract the watercolumn image as numpy array in beam/sample view
        # watercolumn related functions are in the namespace 'watercolumn'
        images_beam_sample.append(ping.watercolumn.get_av())

        # use pingprocessing to retreive an image in depth_x view, 
        # Note(0) this function is currently a bit slow, i'll look into its performance in the future
        # Note(1) this will use additional information about the ping, such as ping.get_geolocation().z and ping.watercolumn.get_beam_crosstrack_angles()
        # Note(2) currently no raytracing, this will be fixed soon
        # Note(3) sorry about the 'from_bottom_xyz=False' your function will fail if you set it to true, will be fixed soon
        # Note(4) I know I am currently using too many namespaces for my functions :-D
        # Note(5) horizontal_res sets the resolution of your image in pixels, 
        #         this function uses backwards mappig so you need to specify the size of your image in pixels
        #         lower pixel count = faster
        #         the vertical resolution will be determined automatically
        # Note (6): the function returns the image (a numpy array) and the extent
        #           If you feed the extent into matplotlib it will be able to transform pixels to meters
        image, extent = pingprocessing.watercolumn.image.make_wci(ping, horizontal_pixels=1024)
        images_depths_y.append((image,extent))
        



In [ ]:
# t
print(f'Info about first beam_sample images: type({type(images_beam_sample[0])}), shape ({images_beam_sample[0].shape})')
# to plot the first image
# plt.imshow(image_beam_sample[0].transpose())

In [ ]:

print(f'Info about first images_depths_y images: type({type(images_depths_y[0])})')
print(f'Info about first images_depths_y images: type({type(images_depths_y[0][0])}), type({type(images_depths_y[0][1])})')
# to plot the first image
# image,extent = images_depths_y[0]
# plt.imshow(image.transpose(), extent=extent)

In [ ]:
vmin = np.nan
vmax = np.nan
for image,extent in tqdm(images_depths_y, 'Get image statistics'):
    vmin = np.nanmin([vmin,np.nanquantile(image,0.02)])
    vmax = np.nanmax([vmax,np.nanquantile(image,0.95)])

print(f'vmin {vmin}, vmax {vmax}')
    
for i,(image,extent) in enumerate(tqdm(images_depths_y, 'Creating images')):
    plt.close('Watercolumn image')
    fig = plt.figure('Watercolumn image')
    ax  = fig.subplots()
    mapable = ax.imshow(image.transpose(), extent=extent, cmap='YlGnBu_r', aspect='equal', vmin=vmin, vmax=vmax)
    ax.set_ylabel('depth (meter)')
    ax.set_xlabel('y (meter')
    fig.colorbar(mapable, shrink=0.25, label = 'Av (dB)')
    fig.set_tight_layout(True)
    #fig.savefig(f'watercolumnimage-{i}.jpg')